In [31]:
import os
import warnings

import netCDF4 as nc
import numpy as np
import xarray as xr

from glob import glob

warnings.filterwarnings('ignore')
os.environ['OMP_NUM_THREADS'] = '1'

In [32]:
output = xr.open_dataset('./output.nc')
output

<xarray.Dataset>
Dimensions:     (interval: 4, percentile: 99, threshold: 10, time: 26, x: 49, y: 40)
Coordinates:
  * percentile  (percentile) int32 1 2 3 4 5 6 7 8 9 ... 92 93 94 95 96 97 98 99
    lons        (x, y) float64 ...
    lats        (x, y) float64 ...
  * time        (time) datetime64[ns] 2020-10-08 ... 2020-10-01T18:00:00
  * threshold   (threshold) float64 0.01 0.1 0.25 0.5 1.0 2.0 3.0 4.0 5.0 6.0
  * interval    (interval) int32 1 6 12 24
Dimensions without coordinates: x, y
Data variables:
    pqpf        (time, interval, percentile, x, y) float32 ...
    qpf         (time, interval, x, y) float32 ...
    pop         (time, interval, x, y) float32 ...
    probx       (time, interval, threshold, x, y) float64 ...
Attributes:
    InitTime:  2020-09-30 12:00:00

In [33]:
lat, lon = output['lats'], output['lons']

In [48]:
ncfilename = './output.ncCustom.nc'

with nc.Dataset(ncfilename, 'w', format='NETCDF4') as ncfile:
    
    ncfile.nx = str(lon.shape[1])
    ncfile.ny = str(lon.shape[0])
    
    ncfile.InitTime = output.attrs['InitTime']
    
    # Lat Lon dimensions and data
    ncfile.createDimension('lon', lon.shape[1])
    ncfile.createDimension('lat', lon.shape[0])
    ncfile.createDimension('time', output['time'].size)
    ncfile.createDimension('interval', output['interval'].size)
    ncfile.createDimension('percentile', output['percentile'].size)
    ncfile.createDimension('threshold', output['threshold'].size)

    lon_nc = ncfile.createVariable('lon', 'f4', ('lat', 'lon'))
    lon_nc.long_name = 'longitude'
    lon_nc.units = 'degrees_east'
    lon_nc.standard_name = 'longitude'
    lon_nc._CoordinateAxisType = 'Lon'

    lat_nc = ncfile.createVariable('lat', 'f4', ('lat', 'lon'))
    lat_nc.long_name = 'latitude'
    lat_nc.units = 'degrees_north'
    lat_nc.standard_name = 'latitude'
    lat_nc._CoordinateAxisType = 'Lat'

    lon_nc[:] = output.lons.values
    lat_nc[:] = output.lats.values
    
    interval = ncfile.createVariable('interval', 'short', ('interval'))
    interval.long_name = 'accumulation interval'
    interval.units = 'hours'
    interval.standard_name = 'interval'
    interval[:] = output['interval'].values.astype(int)
    
    percentile = ncfile.createVariable('percentile', 'short', ('percentile'))
    percentile.long_name = 'accumulation percentile'
    percentile.units = 'none'
    percentile.standard_name = 'percentile'
    percentile[:] = output['percentile'].values.astype(int)

    threshold = ncfile.createVariable('threshold', 'f4', ('threshold'))
    threshold.long_name = 'probabiity of exceedence threshold'
    threshold.units = 'in'
    threshold.standard_name = 'threshold'
    threshold[:] = output['threshold'].values

    # Write variable data
    qpf_nc = ncfile.createVariable('qpf', 'f4', ('time', 'interval', 'lat', 'lon'), fill_value=-9999.0)
    qpf_nc.long_name = 'Deterministic QPF'
    qpf_nc.level = '0'
    qpf_nc.units = 'in'
    qpf_nc[:] = output['qpf'].values
    
    pop_nc = ncfile.createVariable('pop', 'f4', ('time', 'interval', 'lat', 'lon'), fill_value=-9999.0)
    pop_nc.long_name = 'Probability of Precipitation (> 0.01")'
    pop_nc.level = '0'
    pop_nc.units = 'in'
    pop_nc[:] = output['pop'].values
    
    pqpf_nc = ncfile.createVariable('pqpf', 'f4', ('time', 'interval', 'percentile', 'lat', 'lon'), fill_value=-9999.0)
    pqpf_nc.long_name = 'Probabilistic QPF'
    pqpf_nc.level = '0'
    pqpf_nc.units = 'in'
    pqpf_nc[:] = output['pqpf'].values
    
    probx_nc = ncfile.createVariable('probx', 'f4', ('time', 'interval', 'threshold', 'lat', 'lon'), fill_value=-9999.0)
    probx_nc.long_name = 'Probability of Exceedence'
    probx_nc.level = '0'
    probx_nc.units = '%'
    probx_nc[:] = output['probx'].values
        
    print(ncfile)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    nx: 40
    ny: 49
    InitTime: 2020-09-30 12:00:00
    dimensions(sizes): lon(40), lat(49), time(26), interval(4), percentile(99), threshold(10)
    variables(dimensions): float32 lon(lat, lon), float32 lat(lat, lon), int16 interval(interval), int16 percentile(percentile), float32 threshold(threshold), float32 qpf(time, interval, lat, lon), float32 pop(time, interval, lat, lon), float32 pqpf(time, interval, percentile, lat, lon), float32 probx(time, interval, threshold, lat, lon)
    groups: 
